In [1]:
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [2]:
# 讀取 CSV
csv_file = "update_all(yuan).csv"
df = pd.read_csv(csv_file, dtype=str)
# 看前 5 列的所有欄位
#df.iloc[556:705]
df.head()

,href,viewpoint,gmap_location,LOCATION_URL,RQ_URL,GPS_coordinates,coder
0,https://www.taiwan.net.tw/m1.aspx?sNo=0001016&...,蕃薯寮,蕃薯寮休息區,https://www.google.com.tw/maps/place/%E8%95%83...,https://www.google.com.tw/maps/rpc/listugcpost...,"23.7470602,121.5554441",新峻
1,https://www.taiwan.net.tw/m1.aspx?sNo=0001016&...,東河休閒農場,NaN,https://www.google.com.tw/maps/place/%E6%9D%B1...,https://www.google.com.tw/maps/place/%E6%9D%B1...,NaN,新峻
2,https://www.taiwan.net.tw/m1.aspx?sNo=0001016&...,磺溪書院,NaN,https://www.google.com.tw/maps/place/%E7%A3%BA...,https://www.google.com.tw/maps/place/%E7%A3%BA...,NaN,新峻
3,https://www.taiwan.net.tw/m1.aspx?sNo=0001016&...,員貝嶼,NaN,https://www.google.com.tw/maps/place/%E5%93%A1...,https://www.google.com.tw/maps/place/%E5%93%A1...,NaN,新峻
4,https://www.taiwan.net.tw/m1.aspx?sNo=0001016&...,彰化孔子廟,NaN,https://www.google.com.tw/maps/place/%E5%BD%B0...,https://www.google.com.tw/maps/place/%E5%BD%B0...,NaN,新峻


In [3]:
#df['LOCATION_URL'].iloc[556:705]

In [4]:
#df.iloc[556:557]

In [5]:
# 初始化 WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 無頭模式
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [6]:

import requests
import json
import csv
from datetime import datetime, timezone
from time import sleep

# 讀取 CSV 檔案，確保可以讀取所有景點
input_csv = "update_all(yuan).csv"
with open(input_csv, mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    locations = list(reader)  # 讀取所有行，避免只讀取第一行

# **處理每個景點**
for location, url_template in locations:
    print(f"開始爬取景點：{location}")
    
    # 設定輸出檔案名稱
    output_file = f'./CSV/reviews_output_{location}.csv'

    # 變數初始化
    unique_ids = set()
    total_reviews = 0
    reviews_with_comments = 0
    list_2s = []
    list_2s.clear()

    count = 1
    google = ""  # 初始 2s 碼

    # **抓取評論，直到沒有下一個 2s 碼**
    while True:
        list_2s.append({'page': count, 'code_2s': google})

        # **動態替換 URL 內的 2s 參數**
        url = url_template.replace("2s", f"2s{google}")

        response = requests.get(url)
        response.raise_for_status()
        raw_content = response.text.strip()

        # **移除開頭的 ")]}'"**
        if raw_content.startswith(")]}'"):
            raw_content = raw_content[4:]

        data = json.loads(raw_content)

        # **嘗試取得下一個 2s 碼，若無則跳出迴圈**
        try:
            google = data[1].replace('=', '%3D')
        except:
            break

        
        # **解析評論並儲存**
        with open(output_file, mode='a', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            for i in range(10):
                try:
                    user = data[2][i][0][1][4][5][0]
                    user_id = data[2][i][0][1][4][5][3]
                    user_page = data[2][i][0][1][4][2][0]
                    review_id = str(data[2][i][0][0])
                    rating = str(data[2][i][0][2][0][0])
                    timestamp_seconds = data[2][i][0][1][2] // 1000000
                    formatted_date = datetime.fromtimestamp(timestamp_seconds, tz=timezone.utc).strftime('%Y-%m-%d')

                    try:
                        comment = data[2][i][0][2][15][0][0]
                    except (IndexError, TypeError):
                        comment = ""
                    
                    try:
                        language = data[2][i][0][2][14][0]
                    except (IndexError, TypeError):
                        language = ""   
                    
                    try:
                        translated = data[2][i][0][2][15][1][0]
                    except (IndexError, TypeError):
                        translated = ""

                    # **確保 ID 唯一，不重複儲存**
                    if review_id not in unique_ids:
                        writer.writerow([user, user_id, review_id, rating, formatted_date, comment, language, translated,user_page])
                        unique_ids.add(review_id)
                        total_reviews += 1
                        if comment.strip():
                            reviews_with_comments += 1
                except Exception as e:
                    print(f"評論 {i + 1}: 無評論或格式錯誤 ({e})")

        print(f"總共存入 {total_reviews} 筆評論")
        print(f"其中有 {reviews_with_comments} 筆評論有內文")

        count += 1

    print(f"景點 {location} 爬取結束")
    print("=" * 50)

print("所有景點爬取完成")

ValueError: too many values to unpack (expected 2)

In [ ]:
import requests
import json
import csv
from datetime import datetime, timezone
from time import sleep
import random
import os  # 新增這行

# 建立 CSV 輸出目錄
if not os.path.exists('./CSV'):
    os.makedirs('./CSV')

# 讀取 CSV 檔案
input_csv = "update_all(yuan).csv"
with open(input_csv, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)  # 使用 DictReader 來讀取具有標題的 CSV
    locations = []
    for row in reader:
        # 只取需要的欄位：景點名稱和 RQ_URL
        locations.append((row['viewpoint'], row['RQ_URL']))

# 處理每個景點
for location, url_template in locations:
    if not url_template:  # 跳過沒有 URL 的項目
        print(f"跳過景點 {location}：無 URL")
        continue
        
    print(f"開始爬取景點：{location}")
    
    # 設定輸出檔案名稱，移除檔名中可能的特殊字元
    safe_location = "".join(c for c in location if c.isalnum() or c in (' ', '-', '_'))
    output_file = f'./CSV/reviews_output_{safe_location}.csv'

    # 變數初始化
    unique_ids = set()
    total_reviews = 0
    reviews_with_comments = 0
    list_2s = []
    list_2s.clear()

    count = 1
    google = ""  # 初始 2s 碼

    # 建立 CSV 檔案標題
    with open(output_file, mode='w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['使用者名稱', '使用者ID', '評論ID', '評分', '日期', '評論內容', '語言', '翻譯內容', '使用者頁面'])

    # 抓取評論，直到沒有下一個 2s 碼
    while True:
        list_2s.append({'page': count, 'code_2s': google})

        # 動態替換 URL 內的 2s 參數
        url = url_template.replace("2s", f"2s{google}")

        # 加入延遲和錯誤處理
        sleep(random.uniform(1, 3))  # 隨機延遲1-3秒
        
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            raw_content = response.text.strip()
        except requests.exceptions.RequestException as e:
            print(f"請求失敗: {e}")
            break

        # 移除開頭的 ")]}'"
        if raw_content.startswith(")]}'"):
            raw_content = raw_content[4:]

        try:
            data = json.loads(raw_content)
        except json.JSONDecodeError as e:
            print(f"JSON 解析錯誤: {e}")
            break

        # 嘗試取得下一個 2s 碼
        try:
            google = data[1].replace('=', '%3D')
        except:
            break

        # 解析評論並儲存
        with open(output_file, mode='a', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            for i in range(10):
                try:
                    user = data[2][i][0][1][4][5][0]
                    user_id = data[2][i][0][1][4][5][3]
                    user_page = data[2][i][0][1][4][2][0]
                    review_id = str(data[2][i][0][0])
                    rating = str(data[2][i][0][2][0][0])
                    timestamp_seconds = data[2][i][0][1][2] // 1000000
                    formatted_date = datetime.fromtimestamp(timestamp_seconds, tz=timezone.utc).strftime('%Y-%m-%d')

                    try:
                        comment = data[2][i][0][2][15][0][0]
                    except (IndexError, TypeError):
                        comment = ""
                    
                    try:
                        language = data[2][i][0][2][14][0]
                    except (IndexError, TypeError):
                        language = ""   
                    
                    try:
                        translated = data[2][i][0][2][15][1][0]
                    except (IndexError, TypeError):
                        translated = ""

                    # 確保 ID 唯一，不重複儲存
                    if review_id not in unique_ids:
                        writer.writerow([user, user_id, review_id, rating, formatted_date, 
                                      comment, language, translated, user_page])
                        unique_ids.add(review_id)
                        total_reviews += 1
                        if comment.strip():
                            reviews_with_comments += 1
                except Exception as e:
                    print(f"評論 {i + 1}: 無評論或格式錯誤 ({e})")

        print(f"目前共存入 {total_reviews} 筆評論")
        print(f"其中有 {reviews_with_comments} 筆評論有內文")
        print(f"正在處理第 {count} 頁")

        count += 1

    print(f"景點 {location} 爬取結束")
    print(f"最終統計：總共 {total_reviews} 筆評論，{reviews_with_comments} 筆有內文")
    print("=" * 50)

print("所有景點爬取完成")


開始爬取景點：蕃薯寮
目前共存入 10 筆評論
其中有 6 筆評論有內文
正在處理第 1 頁
目前共存入 20 筆評論
其中有 10 筆評論有內文
正在處理第 2 頁
目前共存入 30 筆評論
其中有 14 筆評論有內文
正在處理第 3 頁
目前共存入 40 筆評論
其中有 17 筆評論有內文
正在處理第 4 頁
目前共存入 50 筆評論
其中有 19 筆評論有內文
正在處理第 5 頁
目前共存入 60 筆評論
其中有 24 筆評論有內文
正在處理第 6 頁
目前共存入 70 筆評論
其中有 26 筆評論有內文
正在處理第 7 頁
目前共存入 80 筆評論
其中有 29 筆評論有內文
正在處理第 8 頁
目前共存入 90 筆評論
其中有 34 筆評論有內文
正在處理第 9 頁
目前共存入 100 筆評論
其中有 39 筆評論有內文
正在處理第 10 頁
目前共存入 110 筆評論
其中有 42 筆評論有內文
正在處理第 11 頁
目前共存入 120 筆評論
其中有 45 筆評論有內文
正在處理第 12 頁
目前共存入 130 筆評論
其中有 47 筆評論有內文
正在處理第 13 頁
目前共存入 140 筆評論
其中有 49 筆評論有內文
正在處理第 14 頁
目前共存入 150 筆評論
其中有 53 筆評論有內文
正在處理第 15 頁
目前共存入 160 筆評論
其中有 58 筆評論有內文
正在處理第 16 頁
目前共存入 170 筆評論
其中有 63 筆評論有內文
正在處理第 17 頁
目前共存入 180 筆評論
其中有 67 筆評論有內文
正在處理第 18 頁
目前共存入 190 筆評論
其中有 73 筆評論有內文
正在處理第 19 頁
目前共存入 200 筆評論
其中有 78 筆評論有內文
正在處理第 20 頁
目前共存入 210 筆評論
其中有 82 筆評論有內文
正在處理第 21 頁
目前共存入 220 筆評論
其中有 86 筆評論有內文
正在處理第 22 頁
目前共存入 230 筆評論
其中有 90 筆評論有內文
正在處理第 23 頁
目前共存入 240 筆評論
其中有 93 筆評論有內文
正在處理第 24 頁
目前共存入 250 筆評論
其中有 100 筆評論有內文
正在處理第 25 頁
目前共存入 260 筆評論
其中有 104 筆評論有內文
正在處